In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Explore public HUD REAC data

Author: Jack Vandeleuv

Data files:
* [Public Housing Physical Inspection Scores (2016-2021)](https://www.huduser.gov/portal/datasets/pis.html#2021_data-collapse)
* [Multifamily Physical Inspection Scores (2016-2021)](https://www.huduser.gov/portal/datasets/pis.html#2021_data-collapse)

This notebook will explore the publicly released scores from the U.S. Department of Housing and Urban Development's Real Estate Assessment Center (HUD REAC). These scores are derived from physical inspections on HUD properties.

The goal of this notebook is to aggregate REAC data on a city-by-city basis, so we'll focus on data relevant to that analysis.

# Loading and cleaning

In [6]:
seed = 538

Place the data files in the data directory and load into pandas.

In [7]:
WORKING_DIRECTORY_IN = 'D:/Fire Project/data/raw/reac/'
MULTIFAMILY_FILES = [
    'multifamily_physical_inspection_scores_0321.xlsx',
    'multifamily_physical_inspection_scores_0620.xlsx',
    'multifamily-physical-inspection-scores-2016.xlsx',
    'multifamily-physical-inspection-scores-2018.xlsx',
    'multifamily-physical-inspection-scores-2019.xlsx'
]

PUBLIC_HOUSING_FILES = [
    'public_housing_physical_inspection_scores_0321.xlsx',
    'public_housing_physical_inspection_scores_0620.xlsx',
    'public-housing-physical-inspection-scores-2016.xlsx',
    'public-housing-physical-inspection-scores-2018.xlsx',
    'public-housing-physical-inspection-scores-2019.xlsx'
]

The zipcode columns is alternately named ZIP and ZIPCODE. We'll rename all columns labeled ZIPCODE as ZIP for consistency sake.

In [8]:
public_files = [
    pd.read_excel(WORKING_DIRECTORY_IN + file)
    .rename(columns={'ZIPCODE': 'ZIP'})
    for file in PUBLIC_HOUSING_FILES
]

multi_files = [
    pd.read_excel(WORKING_DIRECTORY_IN + file)
    .rename(columns={'ZIPCODE': 'ZIP'})
    for file in MULTIFAMILY_FILES
]

public = pd.concat(public_files, axis=0)
multi = pd.concat(multi_files, axis=0)

Because the REAC dataset shows the latest inspection for each property, and we're merging data from multiple years, there will be a significant number of duplicates. We'll drop these. We'll define a duplicate as two inspections with the same INSPECTION_ID.

In [9]:
multi = multi.drop_duplicates(subset='INSPECTION_ID')
public = public.drop_duplicates(subset='INSPECTION_ID')


In [10]:
multi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63807 entries, 0 to 27876
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   INSPECTION_ID     63807 non-null  int64  
 1   PROPERTY_ID       63807 non-null  int64  
 2   PROPERTY_NAME     63807 non-null  object 
 3   ADDRESS           62624 non-null  object 
 4   CITY              63807 non-null  object 
 5   CBSA_NAME         58603 non-null  object 
 6   CBSA_CODE         63777 non-null  float64
 7   COUNTY_NAME       63780 non-null  object 
 8   COUNTY_CODE       63781 non-null  float64
 9   STATE_NAME        52362 non-null  object 
 10  STATE_CODE        63807 non-null  object 
 11  ZIP               63787 non-null  float64
 12  LATITUDE          63781 non-null  float64
 13  LONGITUDE         63781 non-null  float64
 14  LOCATION_QUALITY  63781 non-null  object 
 15  INSPECTION_SCORE  63807 non-null  int64  
 16  INSPECTION_DATE   63807 non-null  object

In [11]:
public.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16962 entries, 0 to 6778
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   INSPECTION_ID     16962 non-null  int64  
 1   DEVELOPMENT_ID    16962 non-null  object 
 2   DEVELOPMENT_NAME  16962 non-null  object 
 3   ADDRESS           6639 non-null   object 
 4   CITY              16962 non-null  object 
 5   CBSA_NAME         14634 non-null  object 
 6   CBSA_CODE         16956 non-null  float64
 7   COUNTY_NAME       16956 non-null  object 
 8   COUNTY_CODE       16956 non-null  float64
 9   STATE_NAME        13420 non-null  object 
 10  STATE_CODE        16958 non-null  object 
 11  ZIP               16960 non-null  float64
 12  LATITUDE          16956 non-null  float64
 13  LONGITUDE         16956 non-null  float64
 14  LOCATION_QUALITY  16960 non-null  object 
 15  PHA_CODE          16962 non-null  object 
 16  PHA_NAME          16962 non-null  object 

Let's also look at STATE_NAME and STATE_CODE, which has a lot of null values.

In [12]:
multi.sample(n=5, random_state=seed).loc[:, ['STATE_NAME', 'STATE_CODE']]

,STATE_NAME,STATE_CODE
7548,WV,54
16258,GA,13
3573,VA,51
22460,NaN,MA
4033,AL,1


In [13]:
public.sample(n=5, random_state=seed).loc[:, ['STATE_NAME', 'STATE_CODE']]

,STATE_NAME,STATE_CODE
4103,MD,24
3385,NaN,IN
615,AL,1
3059,VA,51.0
6652,NaN,VA


We also have two different state code formats. We'll combine these into a single column with only the two-letter state codes.

In [14]:
def combine_columns(row):
    if pd.isnull(row['STATE_NAME']):
        return row['STATE_CODE']
    else:
        return row['STATE_NAME']

public['STATE'] = public.apply(combine_columns, axis=1)
multi['STATE'] = multi.apply(combine_columns, axis=1)

public = public.drop(columns=['STATE_NAME', 'STATE_CODE'])
multi = multi.drop(columns=['STATE_NAME', 'STATE_CODE'])

Let's look again to see what nulls are left.

In [15]:
public.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16962 entries, 0 to 6778
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   INSPECTION_ID     16962 non-null  int64  
 1   DEVELOPMENT_ID    16962 non-null  object 
 2   DEVELOPMENT_NAME  16962 non-null  object 
 3   ADDRESS           6639 non-null   object 
 4   CITY              16962 non-null  object 
 5   CBSA_NAME         14634 non-null  object 
 6   CBSA_CODE         16956 non-null  float64
 7   COUNTY_NAME       16956 non-null  object 
 8   COUNTY_CODE       16956 non-null  float64
 9   ZIP               16960 non-null  float64
 10  LATITUDE          16956 non-null  float64
 11  LONGITUDE         16956 non-null  float64
 12  LOCATION_QUALITY  16960 non-null  object 
 13  PHA_CODE          16962 non-null  object 
 14  PHA_NAME          16962 non-null  object 
 15  INSPECTION_SCORE  16962 non-null  int64  
 16  INSPECTION_DATE   16962 non-null  object 

In [16]:
multi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63807 entries, 0 to 27876
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   INSPECTION_ID     63807 non-null  int64  
 1   PROPERTY_ID       63807 non-null  int64  
 2   PROPERTY_NAME     63807 non-null  object 
 3   ADDRESS           62624 non-null  object 
 4   CITY              63807 non-null  object 
 5   CBSA_NAME         58603 non-null  object 
 6   CBSA_CODE         63777 non-null  float64
 7   COUNTY_NAME       63780 non-null  object 
 8   COUNTY_CODE       63781 non-null  float64
 9   ZIP               63787 non-null  float64
 10  LATITUDE          63781 non-null  float64
 11  LONGITUDE         63781 non-null  float64
 12  LOCATION_QUALITY  63781 non-null  object 
 13  INSPECTION_SCORE  63807 non-null  int64  
 14  INSPECTION_DATE   63807 non-null  object 
 15  FIPS_STATE_CODE   11445 non-null  float64
 16  STATE             63807 non-null  object

We have no nulls in CITY and STATE. Let's look at the remaining nulls in ZIP.

In [17]:
print(len(public[public.ZIP.isna()]))
print('Unique states represented among null zipcodes:', public[public.ZIP.isna()].STATE.unique())

2
Unique states represented among null zipcodes: ['PR']


In [18]:
print(len(multi[multi.ZIP.isna()]))
print('Unique states represented among null zipcodes:', multi[multi.ZIP.isna()].STATE.unique())

20
Unique states represented among null zipcodes: ['MA' 'PR' 'NJ' 'VA' 'GA' 'WY' 'TN' 'OH' 'CA' 'DC' 'UT']


Standardize the city names to be upper-case.

In [19]:
multi.CITY = multi.CITY.str.upper()
public.CITY = public.CITY.str.upper()

We have a small number of inspection without zipcode data. We'll leave these for now, and see if we can perform the spatial join without the zipcode data.

We'll convert the datetimes to a standard format.

In [20]:
public['INSPECTION_DATE'] = pd.to_datetime(public['INSPECTION_DATE'], infer_datetime_format=True)
multi['INSPECTION_DATE'] = pd.to_datetime(multi['INSPECTION_DATE'], infer_datetime_format=True)

Let's check to see if we have any cities in the same state with the same name. We'll check this using latitude and longitude.

In [21]:
multi_citystate = multi.loc[:, ['CITY', 'STATE']]
multi_unique_locs = multi_citystate[~multi_citystate.duplicated()].values.tolist()

Let's look at this distribution of inspections by date range.

In [29]:
multi_grouped = multi.groupby(by=multi.INSPECTION_DATE.dt.year).size()
public_grouped = public.groupby(by=public.INSPECTION_DATE.dt.year).size()

side_by_side = pd.concat([multi_grouped, public_grouped], axis=1)
side_by_side.columns = ['Multi', 'Public']
display(side_by_side.sort_index())

,Multi,Public
INSPECTION_DATE,,
2005,NaN,1.0
2012,NaN,5.0
2013,2949.0,336.0
2014,8542.0,1739.0
2015,10762.0,3272.0
2016,9104.0,2424.0
2017,7471.0,3302.0
2018,11856.0,2438.0
2019,11286.0,2566.0


The bulk of inspections in the dataset occurred between 2013 and 2020, with a drop-off when the 2019 pandemic started. 

Export our cleaned data from before 2019. (Reserve the remainder as a validation set.)

In [23]:
multi_13_19 = multi[
    (multi.INSPECTION_DATE.dt.year < 2019) &
    (multi.INSPECTION_DATE.dt.year >= 2013)
]

public_13_19 = public[
    (public.INSPECTION_DATE.dt.year < 2019) &
    (public.INSPECTION_DATE.dt.year >= 2013)
]


In [25]:
WORKING_DIRECTORY_OUT = 'D:/Fire Project/data/processed/'

multi_13_19.to_csv(
    WORKING_DIRECTORY_OUT + 'multi_13-19.csv', 
    sep=',', 
    index=False)

public_13_19.to_csv(
    WORKING_DIRECTORY_OUT + 'public_13-19.csv', 
    sep=',',
    index=False)